In [64]:
%reset -f
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [65]:
# 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows의 경우 '맑은 고딕'
# Mac의 경우: plt.rcParams['font.family'] = 'AppleGothic'
# Linux의 경우 적절한 한글 폰트를 설치해야 함 (예: 'NanumGothic')

# 음수 기호가 깨지는 문제 해결 (minus 폰트 설정)
plt.rcParams['axes.unicode_minus'] = False

In [66]:

# 엑셀 파일 불러오기
# 파일 경로를 지정하세요.
merged_df = pd.read_excel('merged_file.xlsx')  # A 데이터
df = pd.read_excel('df_b.xlsx')  # B 데이터

In [67]:
# 열 이름에서 _x000D_ 제거
merged_df.columns = merged_df.columns.str.replace('_x000D_', '', regex=True)
df.columns = df.columns.str.replace('_x000D_', '', regex=True)

In [68]:
# A데이터 열에서 각 행의 최댓값을 찾아 Max_Value 열로 추가
merged_df['RSB_최종속도'] = merged_df[['RSB_STD_SPEED_20','RSB_STD_SPEED_21','RSB_STD_SPEED_22','RSB_STD_SPEED_23','RSB_STD_SPEED_24','RSB_STD_SPEED_25']].max(axis=1)
merged_df['RSM(LPM)'] = merged_df['RSM(SM)(LPM)']+merged_df['RSM(RM)(LPM)']

In [69]:
# A데이터 'Heat_no'와 '년-월'을 결합한 새로운 열 생성
merged_df['heat_month'] = merged_df['HEATNO_x'].astype(str)+'-'+merged_df['year_month'] 

# B데이터 'Heat_no'와 '년-월'을 결합한 새로운 열 생성
df['heat_month'] = df['HEAT_NO'].astype(str)+'_'+df['압연년월']


In [70]:
# '압연년월' 열을 datetime 형식으로 변환 (B 데이터에 필요)
df['압연년월'] = pd.to_datetime(df['압연년월']).dt.strftime('%Y-%m')  # '날짜' 열에서 년월을 문자열로 변환

# A데이터 열 그룹화
temp_columns = ['균열대.1', '가열대.1','가열대_분','균열대_분','총재로_분','인덕션히터후면_온도']
spd_columns = ['STD_SPEED_8','RSB_최종속도','RSM_SPEED']
cool_columns = ['#9~10ROLL(LPM)','#9~10ROLL(%)', '500(m3/h)', '500(%)', 'HVROLL(%)', 'RSBROLL(Bar)', 'PF/BROLL(%)', 'NTMROLL(%)','RSM(LPM)', 'RSMROLL(%)']

# A데이터 소수점 형식 설정
decimal_formats = {col: '.0f' for col in temp_columns}
decimal_formats.update({col: '.1f' for col in spd_columns})
decimal_formats.update({col: '.1f' for col in cool_columns})

# y_columns 설정
y_columns = temp_columns + spd_columns + cool_columns

# y축 범위를 설정하는 딕셔너리 (각 y값마다 범위를 설정)
y_lim_dict = {
    '균열대.1': (1240, None),
    '가열대.1': (1000, None),  # 다른 변수에 대해 y축 범위 설정
    #'가열대_분': (0, 240),  # 다른 변수에 대해 y축 범위 설정
    #'균열대_분': (0, 240),  # 다른 변수에 대해 y축 범위 설정
    #'총재로_분': (0, 240),  # 다른 변수에 대해 y축 범위 설정
    '인덕션히터후면_온도': (900,1200),  # 다른 변수에 대해 y축 범위 설정
}

# 외경 조건 선택
print("외경 조건을 선택하세요:")
print("1: 외경 조건 없음 (모든 데이터)")
print("2: 모든 고유값 사용")
print("3: 특정값 입력")
choice = input("선택 (1/2/3): ")

# 외경 조건 설정
if choice == '1':
    selected_conditions = None  # 외경 조건 없음 (모든 데이터 선택)
elif choice == '2':
    selected_conditions = df['외경'].unique().tolist()  # 모든 고유값 사용
else:
    specific_values = input("특정값을 쉼표로 구분하여 입력하세요 (예: 100, 150.5, 300): ")
    
    # 소수점 숫자도 포함하여 리스트 생성
    selected_conditions = [float(val) for val in specific_values.split(',')]  # 소수점 값을 허용

# 선택한 외경 조건에 따라 데이터를 반복하여 처리 (여러 외경 값에 대해 그래프 생성)
if selected_conditions is None:
    condition_list = [None]  # 모든 데이터에 대해 단일 조건
else:
    condition_list = selected_conditions  # 고유값 또는 특정값 리스트

# 외경별로 그래프 생성
for condition in condition_list:
    # 조건에 따라 A와 B 데이터 필터링
    if condition is not None:
        filtered_a_df = merged_df[merged_df['지시외경'] == condition]  # A 데이터에서 해당 지시외경 필터링
        filtered_b_df = df[df['외경'] == condition]  # B 데이터에서 해당 외경 필터링
        condition_str = str(condition)  # 외경 번호를 문자열로 변환
    else:
        filtered_a_df = merged_df  # 모든 A 데이터 사용
        filtered_b_df = df  # 모든 B 데이터 사용
        condition_str = "전체"  # 외경이 없으면 "전체"로 설정

    if not filtered_a_df.empty and not filtered_b_df.empty:  # 두 데이터가 모두 있을 때만 그래프 그리기
        # 불량율 계산을 필터링 이후에 수행
        filtered_b_grouped = filtered_b_df.groupby('heat_month').agg({
            '투입중량': 'sum',
            '발생중량': 'sum'
        }).reset_index()
        filtered_b_grouped['불량율'] = (filtered_b_grouped['발생중량'] / filtered_b_grouped['투입중량']) * 100

        # 불량율에 대해 중복된 heat_month 값은 평균으로 집계
        #filtered_b_grouped = filtered_b_grouped.groupby('heat_month').agg({
        # '불량율': 'mean'  # 불량율의 평균을 계산
        #}).reset_index()

        for y_col in y_columns:
            # y값 이름으로 폴더 생성 (슬래시를 언더스코어로 대체)
            output_folder = f'combined_graphs_heat_month_외경별_{y_col.replace("/","_")}'
            os.makedirs(output_folder, exist_ok=True)  # 폴더가 없으면 생성

            # A데이터 'Heat_no_year_month' B데이터'압연년월_Heat'을 기준으로 두 DataFrame 모두 정렬
            filtered_a_df = filtered_a_df.sort_values(by='heat_month')  # A 데이터프레임 정렬
            filtered_b_df = filtered_b_df.sort_values(by='heat_month')  # B 데이터프레임 정렬
                
            fig, ax1 = plt.subplots(figsize=(12, 6))

            # A 그래프 그리기 (왼쪽 y축, y_col에 따라 y값 자동 변경)
            sns.boxplot(x='heat_month', y=y_col, data=filtered_a_df, ax=ax1, color='lightgray')
            ax1.set_xlabel('heat_month')
            ax1.set_ylabel(f'{y_col}', color='gray')
            ax1.tick_params(axis='y', labelcolor='gray')
            plt.xticks(rotation=90)
    
            # 그리드 추가
            plt.grid(True, color='gray',linestyle='--', linewidth=0.4)  # y축 기준으로 그리드 추가

            # A 그래프 중앙값 표시
            medians = filtered_a_df.groupby('heat_month')[y_col].median()
            for xtick, median in zip(ax1.get_xticks(), medians):
                format_str = decimal_formats.get(y_col, '.1f')  # 기본값으로 소수점 1자리 설정
                ax1.text(xtick, median, f'{median:{format_str}}', 
                        ha='center', va='bottom', color='black', fontsize=6)
                
            # A 그래프의 y축 범위 설정
            if y_col in y_lim_dict:
                ax1.set_ylim(y_lim_dict[y_col])

            # B 그래프 그리기 (오른쪽 y축)
            ax2 = ax1.twinx()  # 보조 y축 생성
            sns.lineplot(x='heat_month', y='불량율', data=filtered_b_grouped, ax=ax2, marker='o', color='darkred')
            ax2.set_ylabel('불량율 (%)', color='darkred')
            ax2.tick_params(axis='y', labelcolor='darkred')

            # B 그래프 y축 최소값을 0으로 설정
            ax2.set_ylim(0, None)  # y축 최소값을 0으로, 최대값은 자동 설정

            # 불량률 값 표시
            for index, row in filtered_b_grouped.iterrows():
                ax2.text(row['heat_month'], row['불량율'], f"{row['불량율']:.1f}", 
                        color='darkred', ha='center', va='bottom',fontsize=6)

            # 그래프 제목 및 레이아웃 설정
            plt.title(f'Combined Graph for {y_col} (외경: {condition_str})')

            # x축 레이블이 잘리지 않도록 여백 조정
            plt.subplots_adjust(bottom=0.3)  # 아래쪽 여백을 늘려 x축 레이블 공간 확보
            
            # 그래프 전체 레이아웃 조정 (잘림 방지)
            fig.tight_layout()

            # 범례 추가
            #ax1.legend([f'{y_col}'], loc='upper left', frameon=False)
            #ax2.legend(['불량율(%)'], loc='upper right', frameon=False)

            # 그래프 저장
            plt.savefig(os.path.join(output_folder, f'combined_graph_HEAT_MONTH_{condition_str}_{y_col.replace("/","_")}.png'), dpi=300)
            plt.close()  # 그래프를 닫아 메모리 관리

# 완료 메시지
print("모든 그래프가 저장되었습니다.")

외경 조건을 선택하세요:
1: 외경 조건 없음 (모든 데이터)
2: 모든 고유값 사용
3: 특정값 입력


posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and pos

모든 그래프가 저장되었습니다.


In [71]:
column_list=merged_df.columns.tolist()

print(column_list)

['근_x', '조_x', 'LOTNO_x', '일련번호_x', 'STRNO', 'SERNO', 'HEATNO_x', '회수재여부_x', '사내강종_x', '구사내강종_x', '사내강종(제강)', '강종그룹_x', '지시외경', '투입외경_x', '투입길이_x', '투입중량_x', '입고중량', '주설비구분', '형상', '연마종류', '연마설비', '칸띄움', 'HIWINBUFFER', '소재온도', '수요가', '인식결과', '전송유무', '장입일자', '장입시각', '추출일자', '추출시각', '가열대', '균열대', '계', '균열표준최소', '균열표준최대', '총재로시간표준최소', '총재로시간표준최대', '재로시간(실적-표준)', '압연시작일자_x', '압연시작시각_x', '압연종료일자_x', '압연종료시각_x', '정지시간_x', '정지명_x', '상세내역_x', '가열대.1', '균열대.1', '균열표준', '균열온도(실적-표준)', '예측표면', '예측중심', '불량코드', '사유', 'EVENT', '검사유무', '불량명_x', '결함깊이_x', '중량_x', '원인공정_x', '판정코드', '조치코드_x', '검사일', '검사자', '검사조', '생산품질(Event내역)_x', '병합Lot', '근_y', '조_y', 'LOTNO_y', '분할LON_NO', 'HEATNO_y', '회수재여부_y', '일련번호_y', 'StrNo', 'SerNo', 'Result', '주설비', 'GR종류', '사내강종_y', '구사내강종_y', '사내강종(제강)', '투입외경_y', '투입길이_y', '투입수량', '투입중량_y', '생산수량', '생산중량', '강종그룹_y', '압연외경', '측정외경최소', '측정외경최대', '압연시작일자_y', '압연시작시각_y', '압연종료일자_y', '압연종료시각_y', '정지시간_y', '정지명_y', '상세내역_y', 'STD_SPEED_1', 'STD_SPEED_2', 'STD_SPEED_3', 'STD_SPEE